# Final Exam
`jskyzero` `2018/07/18`

## 分类算法介绍与分析

以下主要介绍随机森林算法与神经网络算法。

### 随机森林

随机森林算法一个包含很多决策树的分类算法，并且通过决策树的“投票”来决定最后的结果。

优点
+ 在数据集上表现良好，相对其他算法有着很大的优势
+ 它能够处理很高维度（feature很多）的数据，并且不用做特征选择
+ 在训练完后，能够给出哪些feature比较重要
+ 在创建随机森林的时候，对generlization error使用的是无偏估计
+ 容易做成并行化方法,训练速度快
+ 在训练过程中，能够检测到feature间的互相影响
+ 实现比较简单

缺点
+ 已经被证明在某些噪音较大的分类或回归问题上会过拟合
+ 对于有不同取值的属性的数据，取值划分较多的属性会对随机森林产生更大的影响，所以随机森林在这种数据上产出的属性权值是不可信的

时间复杂度

+ 假设 n 个样本，m个特征，一棵树的时间复杂度为O(mn logn).
+ 假设用M个树来投票，时间复杂度为O(M(mn log n)).

内存需求

由于每棵树都需要储存，对内存需求极高（在本数据集的具体需求可以参考下方求解过程）

### 神经网络

模仿生物神经元对数据进行函数运算，可以使用大量神经元层级串联层层影响得到最后的结果。

优点

+ 有很强的非线性拟合能力，可映射任意复杂的非线性关系
+ 学习规则和迭代规则简单，便于计算机实现
+ 具有很强的鲁棒性、记忆能力、非线性映射能力以及强大的自学习能力

缺点

+ 没有特别直观的解释推理过程和推理依据的方法

+ 当数据不充分的时候，效果相对较差

+ 把数据完全理解为数字，将拟合过程变成纯数学问题，有丢失隐信息的风险


时间复杂度

假设为一个简单的三层BP神经网络，每层神经元数量分别为n1，n2，n3。（输入层和最终输出层结点数量(n1和n3)是确定）
+ 前馈计算，两次矩阵乘法（实际上是向量和矩阵相乘）分别要进行n1 \* n2 和 n2 \* n3次计算，算时间复杂度是O(n1 \* n2 + n2 \* n3) = O(n2)。
+ 反向传播时，假设总共有m个训练样本，每个样本只训练一次，时间复杂度应该是O(m*n2)。


内存需求

相较随机森林内存占用少了很多，主要是可以分次迭代每次只需要储存当次的数据。


## 求解过程

### 预处理过程

将数据处理为方便操作的形式。

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import h5py
import gc
import shelve

%matplotlib inline


# set cofig
FILE_PATH = "D:\\Downloads\\"
SAMPLE_FILE_NAME = "submission_sample.csv"
TRAIN_FILE_NAME = "train_data.mat"
TEST_FILE_NAME = "test_data_raw.mat"
OUT_FILE_NAME = "out.csv"
SHELVE_FILE_NAME = "shelve"

# read train file
train_feat = h5py.File(FILE_PATH + TRAIN_FILE_NAME)['train_feat'][:]
train_label = h5py.File(FILE_PATH + TRAIN_FILE_NAME)['train_label'][:]

X = np.transpose(np.array(train_feat))
Y = np.array(train_label)[0]

print("X.shape = {}".format(X.shape))
print("Y.shape = {}".format(Y.shape))

### 可选的降维和数据清洗

因为后面试过具体的降维的度比较难把握所以放弃进行降维，清洗同理。

In [ ]:
# PCA
# from sklearn.decomposition import PCA
# pca = PCA(n_components=1000)
# pca.fit(X)
# X = pca.transform(X)

### 分割数据集
方便本地验证，如果需要最终提交的使用下面一个版本即可。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

# with shelve.open(FILE_PATH + SHELVE_FILE_NAME) as db:
#     db['X_train'] = X_train
#     db['X_test'] = X_test
#     db['y_train'] = y_train
#     db['y_test'] = y_test


with shelve.open(FILE_PATH + SHELVE_FILE_NAME) as db:
    db['X_train'] = X
    db['X_test'] = ""
    db['y_train'] = Y
    db['y_test'] = ""
gc.collect()

### 再次读取数据

因为前面预处理使用内存较大所以可以分步执行减低内存需求。

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import h5py
import gc
import shelve

%matplotlib inline


# set cofig
FILE_PATH = "D:\\Downloads\\"
SAMPLE_FILE_NAME = "submission_sample.csv"
TRAIN_FILE_NAME = "train_data.mat"
TEST_FILE_NAME = "test_data_raw.mat"
OUT_FILE_NAME = "out.csv"
SHELVE_FILE_NAME = "shelve"


# 读取数据
with shelve.open(FILE_PATH + SHELVE_FILE_NAME) as db:
    X_train= db['X_train']
    X_test = db['X_test']
    y_train = db['y_train']
    y_test = db['y_test']

### 随机森林

需要内存16G，I7 7700短期内可跑出，正确率大概0.14

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(criterion="gini",
#                              max_features=20,
# #                              max_depth=4,
#                              n_estimators=140,
#                              oob_score=True,
#                              n_jobs=8,
#                              random_state=0)
# clf.fit(X_train, y_train)

### 神经网络

相同配置，迭代大概需要数个小时，各种调整参数吼效果0.25+

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data

# X_test = scaler.transform(X_test)


from sklearn.neural_network import MLPClassifier
size = [1000]
clf = MLPClassifier(hidden_layer_sizes=size,
                    activation='tanh',
                    solver='sgd',
#                     alpha=0.0001,
                    max_iter=100,
                    verbose=True,
#                     tol=0.001
                   )
clf.fit(X_train, y_train)

### 测试集效果
可以观察是否过拟合，随即森林会出现较为明显的过拟合，多次迭代后升级网络在训练集也能达到1.0的正确率

In [ ]:
print("train error {}".format(clf.score(X_train, y_train)))
# print("test error {}".format(clf.score(X_test, y_test)))

### 提交
由于数据读取的问题，找个编辑器手动去掉浮点数的小数部分即可。

In [ ]:
test = h5py.File(FILE_PATH + TEST_FILE_NAME)['test_feat']
test = np.transpose(np.array(test))
print("test.shape = {}".format(test.shape))
test = scaler.transform(test) 
Y_test = clf.predict(test)
sample_submission =  pd.read_csv(FILE_PATH + SAMPLE_FILE_NAME, sep=',')
print("submisstion data shape = {}".format(sample_submission.shape))
sample_submission.iloc[:, 1:2] = Y_test
sample_submission.to_csv(FILE_PATH + "0" + OUT_FILE_NAME, sep=',', index=False)